In [ ]:
import re 
import time
import pandas as pd
from selenium import webdriver
import requests

from bs4 import BeautifulSoup
# 게임명 장르 발매일 선호도(%) 리뷰수 조사

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}

def steam_crawl_contents(start, end):   # 몇 번째부터 몇 번째 태그의 정보를 불러올건지
    
    count = 0
    game_df = pd.DataFrame([], columns=['게임명', '장르', '태그', '발매일', '선호도(%)', '리뷰수'])

    url = 'https://store.steampowered.com/search/?filter=globaltopsellers&os=win'   # top seller url
    
    driver = webdriver.Chrome('chromedriver.exe')
    driver.maximize_window()
    driver.get(url)
    driver.implicitly_wait(10)
    
    
    # 스크롤 내리기
    for i in range(12000):   # 괄호 안에 숫자만큼 스크롤 내리기 반복
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
    time.sleep(1)
    
    res = driver.page_source
    soup = BeautifulSoup(res, 'lxml')
    p = re.compile("\d*\.?\d+")
    driver.close()
    
    game_lst = soup.find('div', attrs={'id':'search_resultsRows'}).find_all('a')
    
    for i in range(start, end + 1):
        
        # 몇 개의 게임이나 진행됐는지 카운트
        count+=1
        print(count)
        
        detail_url = game_lst[i]['href']     # 게임리스트 중에 하나씩 꺼내서 탐색
        res_detail = requests.get(detail_url, headers=headers)
        soup_detail = BeautifulSoup(res_detail.text, 'lxml')
        
        # 게임명
        if soup_detail.find('div', attrs={'class':'apphub_AppName'}) == None:
            continue
        else:
            game_name = soup_detail.find('div', attrs={'class':'apphub_AppName'}).get_text()
            game_name = game_name.replace('®', '')
            print(game_name)

        # 장르
        genre_lst = []
        genres = soup_detail.find('div', attrs={'id':'genresAndManufacturer'}).find_all('a')
        for genre in genres:
            if 'genre' in genre['href']:
                genre_lst.append(genre.get_text())
            else:
                pass
                    
        # 태그
        if soup_detail.find('div', attrs={'class':'glance_tags popular_tags'}) == None:
            continue
        else:
            tag_lst = []
            tags = soup_detail.find('div',attrs={'class':'glance_tags popular_tags'}).find_all('a')
            for tag in tags:
                tag_lst.append(tag.get_text().strip())
            
        # 발매일
        if soup_detail.find('div', attrs={'class':'date'}) == None:
            date = ''
        else:
            date = soup_detail.find('div', attrs={'class':'date'}).get_text()

        # 선호도(%)
        if soup_detail.find('div', attrs={'class':'user_reviews_summary_bar'}) == None:
            continue
        else:
            positive_percent = soup_detail.find('div', attrs={'class':'user_reviews_summary_bar'}).find_all('span')[0]
            if p.findall(positive_percent['data-tooltip-html']) == []:
                continue
            else:
                positive_percent = p.findall(positive_percent['data-tooltip-html'])[0]
        
        # 리뷰수    
        review = soup_detail.find('div', attrs={'class':'summary_section'}).find_all('span')[-1]
        review = re.sub(r'[^0-9]', '', review.get_text())
    
        # 데이터프레임 생성 및 합치기
        game_sr = pd.DataFrame([[game_name, genre_lst, tag_lst, date, positive_percent, review]], columns=['게임명', '장르', '태그', '발매일', '선호도(%)', '리뷰수'])
        game_df = pd.concat((game_df, game_sr))
#         print(game_df)
        time.sleep(1)
    
        if count % 100 == 0:
            game_df.to_csv('게임리스트(global_topseller12).csv', mode='w', encoding='utf-8-sig', index=False)
            time.sleep(1)
    print('complete')
    
steam_crawl_contents(16501,16600)